In [20]:
!pip install jsonlines

In [0]:
import json
import jsonlines
import numpy as np
import random
from copy import deepcopy

# Create basic dataset
with open('task1_headline_ABSA_train.json', 'r') as f:
    array1 = json.load(f)
with open('task1_post_ABSA_train.json', 'r') as f:
    array2 = json.load(f)
array = dict(array1, **array2)
data_array = {}

temp_array_base = deepcopy(array)
with jsonlines.open('task_train.jsonl', mode='w') as writer:
    idx_list = random.sample(array.keys(), 944)
    for idx in idx_list:
        final = {}
        a = array[idx]
        final['question'] = a['sentence']
        info = a['info'][0]
        final['passage'] = info['aspects'].strip('[').strip(']').strip('\'')
        final['idx'] = idx
        if float(info['sentiment_score']) > 0:
            final['label'] = "1"
        else:
            final['label'] = "0"
        data_array[idx] = final
        writer.write(final)

        del temp_array_base[idx]

with jsonlines.open('task_val.jsonl', mode='w') as writer:
    
    idx_list = temp_array_base.keys()
    for idx in idx_list:
        final = {}
        a = array[idx]
        final['question'] = a['sentence']
        info = a['info'][0]
        final['passage'] = info['aspects'].strip('[').strip(']').strip('\'')
        final['idx'] = idx
        if float(info['sentiment_score']) > 0:
            final['label'] = "1"
        else:
            final['label'] = "0"
        data_array[idx] = final
        writer.write(final)

# Only level one
temp_array_only_one = deepcopy(data_array)
with jsonlines.open('task_train_level_one.jsonl', mode='w') as writer:
    idx_list = random.sample(data_array.keys(), 944)
    for idx in idx_list:
        level_one_aspect = temp_array_only_one[idx]['passage'].split('/')[0]
        temp_array_only_one[idx]['passage'] = level_one_aspect
        writer.write(temp_array_only_one[idx])
        del temp_array_only_one[idx]

with jsonlines.open('task_val_level_one.jsonl', mode='w') as writer:
    idx_list = temp_array_only_one.keys()
    for idx in idx_list:
        level_one_aspect = temp_array_only_one[idx]['passage'].split('/')[0]
        temp_array_only_one[idx]['passage'] = level_one_aspect
        writer.write(temp_array_only_one[idx])

# Only level two
temp_array_only_two = deepcopy(data_array)
with jsonlines.open('task_train_level_two.jsonl', mode='w') as writer:
    idx_list = random.sample(data_array.keys(), 944)
    for idx in idx_list:
        sentence_aspect = temp_array_only_two[idx]['passage'].split('/')
        if len(sentence_aspect) > 1:
            level_two_aspect = sentence_aspect[1]
        else:
            level_two_aspect = sentence_aspect[0]
        temp_array_only_two[idx]['passage'] = level_two_aspect
        writer.write(temp_array_only_two[idx])
        del temp_array_only_two[idx]

with jsonlines.open('task_val_level_two.jsonl', mode='w') as writer:
    idx_list = temp_array_only_two.keys()
    for idx in idx_list:
        sentence_aspect = temp_array_only_two[idx]['passage'].split('/')
        if len(sentence_aspect) > 1:
            level_two_aspect = sentence_aspect[1]
        else:
            level_two_aspect = sentence_aspect[0]
        temp_array_only_two[idx]['passage'] = level_two_aspect
        writer.write(temp_array_only_two[idx])

# Create random aspect dataset
aspect_list = []
level_num = 0
with open('task_train.jsonl', 'r') as f:
    for item in jsonlines.Reader(f):
        aspect_list.append(item['passage'].split('/'))
        temp = len(aspect_list[-1])
        if temp > level_num:
            level_num = temp
with open('task_val.jsonl', 'r') as f:
    for item in jsonlines.Reader(f):
        aspect_list.append(item['passage'].split('/'))
        temp = len(aspect_list[-1])
        if temp > level_num:
            level_num = temp
aspect_set = []
for i in range(level_num):
    aspect_set.append(set())
for item in aspect_list:
    for i in range(len(item)):
        aspect_set[i].add(item[i])
for i in range(level_num):
    aspect_set[i] = list(aspect_set[i])

# All levels random
temp_array_random_all = deepcopy(data_array)
with jsonlines.open('task_train_random_all.jsonl', mode='w') as writer:
    idx_list = random.sample(data_array.keys(), 944)
    for idx in idx_list:
        rand_aspect_list = []
        rand_level = np.random.randint(1, level_num)
        for i in range(rand_level):
            rand_ind = np.random.randint(0, len(aspect_set[i]))
            rand_aspect_list.append(aspect_set[i][rand_ind])
        rand_aspect = "/".join(rand_aspect_list)
        temp_array_random_all[idx]['passage'] = rand_aspect
        writer.write(temp_array_random_all[idx])
        del temp_array_random_all[idx]

with jsonlines.open('task_val_random_all.jsonl', mode='w') as writer:
    idx_list = temp_array_random_all.keys()
    for idx in idx_list:
        rand_aspect_list = []
        rand_level = np.random.randint(1, level_num)
        for i in range(rand_level):
            rand_ind = np.random.randint(0, len(aspect_set[i]))
            rand_aspect_list.append(aspect_set[i][rand_ind])
        rand_aspect = "/".join(rand_aspect_list)
        temp_array_random_all[idx]['passage'] = rand_aspect
        writer.write(temp_array_random_all[idx])

    

In [28]:
with jsonlines.open('task_train.jsonl') as reader:
    for obj in reader:
        print(obj['passage'])
        break

Corporate/Sales
